# Cryptarithms: Send More Money

The July, 1924, issue of the famous British magazine *The Strand* included a word puzzle by Henry E. Dudeney in his regular contribution "Perplexities".  The puzzle is to assign a unique digit to each letter appearing in the equation

        S E N D
      + M O R E
    = M O N E Y

such that the arithmetic equation is satisfied, and the leading digit for M is non-zero. There are [many more examples](http://cryptarithms.awardspace.us/puzzles.html) of these puzzles, but this is perhaps the most well-known.

This notebook demonstrates a solution to this puzzle using Pyomo disjuctions and the `gecode` solver, a constraint solving package written in C++. This same puzzle is used in the `gecode` documentation, so this notebook may provide a useful contrast between Pyomo modeling and use of a native C++ API.

In [ ]:
# install Pyomo and solvers for Google Colab
import sys
if "google.colab" in sys.modules:
    !wget -N -q https://raw.githubusercontent.com/jckantor/MO-book/main/tools/install_on_colab.py 
    %run install_on_colab.py

installing pyomo . pyomo installed
installing and testing solvers ...
.. glpk
.. ipopt
.. gecode
.. bonmin
.. couenne
.. gurobi_direct
.. cbc
installation and testing complete


## Modeling and Solution

There are several possible approaches to modeling this puzzle in Pyomo. 

[One approach](https://stackoverflow.com/questions/67456379/pyomo-model-constraint-programming-for-sendmore-money-task)  would be to using a matrix of binary variables $x_{a,d}$ indexed by letter $a$ and digit $d$ such that $x_{a,d} = 1$ designates the corresponding assignment. The problem constraints can then be implemented by summing the binary variables along the two axes. The arithmetic constraint becomes a more challenging.

[Another approach](https://www.gecode.org/doc/6.0.1/MPG.pdf) is to use Pyomo integer variables indexed by letters, then setup an linear expression to represent the puzzle following the scheme

                 1000 S + 100 E + 10 N + D
               + 1000 M + 100 O + 19 R + E
    == 10000 M + 1000 O + 100 N + 10 E + Y

The requirement that no two letters be assigned the same digit could, in principle, be implemented with an inequality constraint.  A more robust approach is to use a disjunction. Letting $n_a$ and $n_b$ denote the integers assigned to letters $a$ and $b$, the disjuction becomes

$$
\begin{align*}
\begin{bmatrix}n_a \lt n_b\end{bmatrix} 
\veebar &
\begin{bmatrix}n_b \lt n_a\end{bmatrix} 
& \forall a \lt b
\end{align*}$$

Here we use the `gecode` solver which is well suited to problems integer, logic, and binary variables and disjunctive constraints.


In [ ]:
import pyomo.environ as pyo
import pyomo.gdp as gdp

m = pyo.ConcreteModel()

m.LETTERS = pyo.Set(initialize=['S', 'E', 'N', 'D', 'M', 'O', 'R', 'Y'])
m.n = pyo.Var(m.LETTERS, domain=pyo.Integers, bounds=(0, 9))

@m.Constraint()
def message(m):
    return               1000*m.n['S'] + 100*m.n['E'] + 10*m.n['N'] + m.n['D'] \
                       + 1000*m.n['M'] + 100*m.n['O'] + 10*m.n['R'] + m.n['E'] \
     == 10000*m.n['M'] + 1000*m.n['O'] + 100*m.n['N'] + 10*m.n['E'] + m.n['Y']

# leading digit must be non-zero
@m.Constraint()
def leading_digit_nonzero(m):
    return m.n['M'] >= 1

# assign a different number to each letter
@m.Disjunction(m.LETTERS, m.LETTERS)
def unique_assignment(m, a, b):
    if a < b:
        return [m.n[a] >= m.n[b] + 1,
                m.n[b] >= m.n[a] + 1]
    else:
        return gdp.Disjunction.Skip

pyo.TransformationFactory('gdp.bigm').apply_to(m)
solver = pyo.SolverFactory('gecode')
solver.solve(m)

print(f"    {int(m.n['S']())} {int(m.n['E']())} {int(m.n['N']())} {int(m.n['D']())}")
print(f"  + {int(m.n['M']())} {int(m.n['O']())} {int(m.n['R']())} {int(m.n['E']())}")
print(f"  ---------")
print(f"= {int(m.n['M']())} {int(m.n['O']())} {int(m.n['N']())} {int(m.n['E']())} {int(m.n['Y']())}")

    9 5 6 7
  + 1 0 8 5
  ---------
= 1 0 6 5 2


## Suggest exercises

1. Pyomo includes a logic-based solver `GDPopt` for generalized disjunctive programming problems. Implement and test `GDPopt` using various combinations of solution strategies and MIP solvers, and compare the performance to the constraint solver `gecode`.

2. There are [many more examples](http://cryptarithms.awardspace.us/puzzles.html) this puzzles. Refactor this code and create a function that can be used to solve generic puzzles of this type.